![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FPipelines&file=Vertex+AI+Pipelines+-+Managing+Pipeline+Jobs.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Pipelines/Vertex%20AI%20Pipelines%20-%20Managing%20Pipeline%20Jobs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FPipelines%2FVertex%2520AI%2520Pipelines%2520-%2520Managing%2520Pipeline%2520Jobs.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Pipelines/Vertex%20AI%20Pipelines%20-%20Managing%20Pipeline%20Jobs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Pipelines/Vertex%20AI%20Pipelines%20-%20Managing%20Pipeline%20Jobs.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---
This is part of a [series of notebook based workflows](./readme.md) that teach all the ways to use pipelines within Vertex AI. The suggested order and description/reason is:

|Link To Section|Notebook Workflow|Description|
|---|---|---|
||[Vertex AI Pipelines - Start Here](./Vertex%20AI%20Pipelines%20-%20Start%20Here.ipynb)|What are pipelines? Start here to go from code to pipeline and see it in action.|
||[Vertex AI Pipelines - Introduction](./Vertex%20AI%20Pipelines%20-%20Introduction.ipynb)|Introduction to pipelines with the console and Vertex AI SDK|
||[Vertex AI Pipelines - Components](./Vertex%20AI%20Pipelines%20-%20Components.ipynb)|An introduction to all the ways to create pipeline components from your code|
||[Vertex AI Pipelines - IO](./Vertex%20AI%20Pipelines%20-%20IO.ipynb)|An overview of all the type of inputs and outputs for pipeline components|
||[Vertex AI Pipelines - Control](./Vertex%20AI%20Pipelines%20-%20Control.ipynb)|An overview of controlling the flow of exectution for pipelines|
||[Vertex AI Pipelines - Secret Manager](./Vertex%20AI%20Pipelines%20-%20Secret%20Manager.ipynb)|How to pass sensitive information to pipelines and components|
||[Vertex AI Pipelines - GCS Read and Write](./Vertex%20AI%20Pipelines%20-%20GCS%20Read%20and%20Write.ipynb)|How to read/write to GCS from components, including container components.|
||[Vertex AI Pipelines - Scheduling](./Vertex%20AI%20Pipelines%20-%20Scheduling.ipynb)|How to schedule pipeline execution|
||[Vertex AI Pipelines - Notifications](./Vertex%20AI%20Pipelines%20-%20Notifications.ipynb)|How to send email notification of pipeline status.|
||[Vertex AI Pipelines - Management](./Vertex%20AI%20Pipelines%20-%20Management.ipynb)|Managing, Reusing, and Storing pipelines and components|
||[Vertex AI Pipelines - Testing](./Vertex%20AI%20Pipelines%20-%20Testing.ipynb)|Strategies for testing components and pipeliens locally and remotely to aide development.|
|_**This Notebook**_|[Vertex AI Pipelines - Managing Pipeline Jobs](./Vertex%20AI%20Pipelines%20-%20Managing%20Pipeline%20Jobs.ipynb)|Manage runs of pipelines in an environment: list, check status, filtered list, cancel and delete jobs.|


To discover these notebooks as part of an introduction to MLOps orchestration [start here](./readme.md).  To read more about MLOps also check out [the parent folder](../readme.md).

---

# Vertex AI Pipelines - Managing Jobs

Vertex AI Pipeline Jobs are runs of a pipeline.  These can be directly run by a user, started by API, or scheduled.  Withing Vetex AI a project can have many jobs running at any time and a history of all past jobs.  This workflow shows how to review and manage the jobs in an enviornment using the Python SDK.  For custom metrics in Cloud Logging check out [this helpful page](https://cloud.google.com/vertex-ai/docs/pipelines/metrics).

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.51.0'),
    ('kfp', 'kfp'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
EXPERIMENT = 'pipeline-jobstate'
SERIES = 'mlops'

# gcs bucket
GCS_BUCKET = PROJECT_ID

Packages

In [55]:
import os, datetime, collections

from google.cloud import aiplatform
import kfp

In [9]:
kfp.__version__

'2.11.0'

In [10]:
aiplatform.__version__

'1.78.0'

Clients

In [11]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

parameters:

In [12]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"

In [13]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [14]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Make A Simple Pipeline For Testing/Illustrating

In [15]:
@kfp.dsl.component(
    base_image = "python:3.11"
)
def waiter(wait_time: int) -> dict:
    import time
    
    time.sleep(wait_time)
    
    return

In [16]:
kfp.compiler.Compiler().compile(
    pipeline_func = waiter,
    package_path = f'{DIR}/{SERIES}-{EXPERIMENT}.yaml',
    pipeline_name = f"{SERIES}-{EXPERIMENT}-component"
)

## Executing Pipeline Jobs And Monitoring Directly

### Execute A PipelineJob with `.run()`

This method runs and locks the local context while it waits on the job to end.  It continually updates the state of the job in the output to the local session.

In [17]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-component",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        wait_time = 10
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = False # True (enabled), False (disable), None (defer to component level caching) 
)

In [18]:
job_run = pipeline_job.run(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214124
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214124')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-jobstate-component-20250213214124?project=1026793852137
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214124 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214124 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214124 current s

#### Understanding the Final State Of The PipelineJob

Notice that the job continually reported state for the duration of running.

In [19]:
pipeline_job.state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [20]:
pipeline_job.state.value

4

In [21]:
pipeline_job.state.name

'PIPELINE_STATE_SUCCEEDED'

In [22]:
type(pipeline_job.state)

<enum 'PipelineState'>

In [23]:
pipeline_job.done()

True

https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.PipelineState

In [24]:
aiplatform.gapic.PipelineState.__members__

mappingproxy({'PIPELINE_STATE_UNSPECIFIED': <PipelineState.PIPELINE_STATE_UNSPECIFIED: 0>,
              'PIPELINE_STATE_QUEUED': <PipelineState.PIPELINE_STATE_QUEUED: 1>,
              'PIPELINE_STATE_PENDING': <PipelineState.PIPELINE_STATE_PENDING: 2>,
              'PIPELINE_STATE_RUNNING': <PipelineState.PIPELINE_STATE_RUNNING: 3>,
              'PIPELINE_STATE_SUCCEEDED': <PipelineState.PIPELINE_STATE_SUCCEEDED: 4>,
              'PIPELINE_STATE_FAILED': <PipelineState.PIPELINE_STATE_FAILED: 5>,
              'PIPELINE_STATE_CANCELLING': <PipelineState.PIPELINE_STATE_CANCELLING: 6>,
              'PIPELINE_STATE_CANCELLED': <PipelineState.PIPELINE_STATE_CANCELLED: 7>,
              'PIPELINE_STATE_PAUSED': <PipelineState.PIPELINE_STATE_PAUSED: 8>})

### Execute A PipelineJob with `.submit()` and `.wait()`

This method starts the job and returns the context to the local session without waiting on the job to complete.  The wait method can be used to force the session to wait on the job to end.

In [25]:
pipeline_job2 = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-component",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        wait_time = 10
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = False # True (enabled), False (disable), None (defer to component level caching) 
)

In [26]:
pipeline_job2.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-jobstate-component-20250213214746?project=1026793852137


Notice that the job returns to the local context after it is sumitted.

In [28]:
type(pipeline_job2)

google.cloud.aiplatform.pipeline_jobs.PipelineJob

In [29]:
pipeline_job2.state

<PipelineState.PIPELINE_STATE_RUNNING: 3>

In [30]:
pipeline_job2.done()

False

In [31]:
pipeline_job2.wait()

PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746


In [32]:
pipeline_job2.state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [33]:
pipeline_job2.done()

True

In [34]:
pipeline_job2.resource_name

'projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746'

### Execute A PipelineJob And Monitor it Directly

Similar to the previous example, execute a PipelineJob with `.submit()`.  Then `.get()` the job separately (as though in a different session at a different time) and directly monitor its status.

In [38]:
datetime.datetime.now().strftime("%m%d%Y-%H%M%S")

'02132025-215722'

In [39]:
job_id = f'{SERIES}-{EXPERIMENT}-component-{datetime.datetime.now().strftime("%m%d%Y-%H%M%S")}'

pipeline_job3 = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-component",
    job_id = job_id,
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        wait_time = 100
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = False # True (enabled), False (disable), None (defer to component level caching) 
)

In [40]:
pipeline_job3.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-jobstate-component-02132025-215756?project=1026793852137


In [42]:
retrieve_job3 = aiplatform.PipelineJob.get(resource_name = job_id)

In [43]:
pipeline_job3.resource_name

'projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756'

In [44]:
retrieve_job3.resource_name

'projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756'

In [45]:
retrieve_job3.done()

False

In [46]:
retrieve_job3.state

<PipelineState.PIPELINE_STATE_RUNNING: 3>

In [47]:
retrieve_job3.wait()

PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756


In [48]:
retrieve_job3.state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [49]:
pipeline_job3.state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

---
## Listing Pipeline Jobs

### All Runs

In [59]:
jobs_list = aiplatform.PipelineJob.list(filter = None, enable_simple_view = True)

By enabling simple view, the PipelineJob resources returned from this method will not contain all fields.


In [60]:
len(jobs_list)

384

In [61]:
jobs_list[0:3]

 resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-02132025-215756,
 resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214746,
 resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213214124]

In [62]:
jobs_list[0].state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [63]:
states = [job.state.name for job in jobs_list]

In [64]:
collections.Counter(states)

Counter({'PIPELINE_STATE_SUCCEEDED': 196,
         'PIPELINE_STATE_FAILED': 185,
         'PIPELINE_STATE_CANCELLED': 3})

### Named Runs: Filter By `display_name`

This gets all the runs for a pipeline by using the pipelines display name as the filter.

In [65]:
jobs_list = aiplatform.PipelineJob.list(
    filter = f'display_name = "{SERIES}-{EXPERIMENT}-component"', 
    enable_simple_view = True
)

By enabling simple view, the PipelineJob resources returned from this method will not contain all fields.


In [66]:
len(jobs_list)

6

In [67]:
jobs_list[0].state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

### Query Runs: Filter By `display_name` and `state`

Get all successful runs of a pipeline, and order by creation time descending.

In [70]:
jobs_list = aiplatform.PipelineJob.list(
    filter = f'display_name = "{SERIES}-{EXPERIMENT}-component" AND state = "PIPELINE_STATE_SUCCEEDED"', 
    order_by = 'create_time desc',
    enable_simple_view = True
)

By enabling simple view, the PipelineJob resources returned from this method will not contain all fields.


In [71]:
len(jobs_list)

6

In [74]:
[(job.name, job.create_time.strftime("%Y-%m-%d %H:%M:%S")) for job in jobs_list]

[('mlops-pipeline-jobstate-component-02132025-215756', '2025-02-13 21:58:03'),
 ('mlops-pipeline-jobstate-component-20250213214746', '2025-02-13 21:47:47'),
 ('mlops-pipeline-jobstate-component-20250213214124', '2025-02-13 21:41:24'),
 ('mlops-pipeline-jobstate-component-20250212193454', '2025-02-12 19:35:07'),
 ('mlops-pipeline-jobstate-component-20250212192434', '2025-02-12 19:24:45'),
 ('mlops-pipeline-jobstate-component-20250212175926', '2025-02-12 17:59:46')]

### Active Runs: Filter by `state`

In [75]:
pipeline_job4 = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-component",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        wait_time = 1000
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = False # True (enabled), False (disable), None (defer to component level caching) 
)

In [76]:
pipeline_job4.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-jobstate-component-20250213223849?project=1026793852137


In [77]:
jobs_list = aiplatform.PipelineJob.list(
    filter = f'display_name = "{SERIES}-{EXPERIMENT}-component" AND state = "PIPELINE_STATE_RUNNING"', 
    order_by = 'create_time desc',
    enable_simple_view = True
)

By enabling simple view, the PipelineJob resources returned from this method will not contain all fields.


In [78]:
[(job.name, job.create_time.strftime("%Y-%m-%d %H:%M:%S")) for job in jobs_list]

[('mlops-pipeline-jobstate-component-20250213223849', '2025-02-13 22:38:50')]

In [79]:
jobs_list[0].done()

False

In [117]:
jobs_list[0].create_time

DatetimeWithNanoseconds(2025, 2, 13, 22, 38, 50, 258720, tzinfo=datetime.timezone.utc)

### Jobs In Date Range

In [122]:
jobs_list = aiplatform.PipelineJob.list(
    filter = 'create_time >= "2024-06-01T00:00:00Z" AND create_time < "2024-07-01T00:00:00Z"'
)

In [123]:
len(jobs_list)

33

In [124]:
collections.Counter([job.state.name for job in jobs_list])

Counter({'PIPELINE_STATE_FAILED': 19, 'PIPELINE_STATE_SUCCEEDED': 14})

---
## Job Canceling, Deleting, And Cloning

### Stop Jobs with `.cancel()`

In [80]:
jobs_list[0].done()

False

In [81]:
jobs_list[0].state

<PipelineState.PIPELINE_STATE_RUNNING: 3>

In [82]:
jobs_list[0].cancel()

In [83]:
jobs_list[0].state

<PipelineState.PIPELINE_STATE_CANCELLING: 6>

In [84]:
jobs_list[0].done()

False

In [85]:
jobs_list[0].wait()

PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849


In [86]:
jobs_list[0].done()

True

In [87]:
jobs_list[0].state

<PipelineState.PIPELINE_STATE_CANCELLED: 7>

### Delete Jobs with `.delete()`

In [89]:
jobs_list[0].delete()

Deleting PipelineJob : projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849
PipelineJob deleted. . Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849
Deleting PipelineJob resource: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849
Delete PipelineJob backing LRO: projects/1026793852137/locations/us-central1/operations/3691838885597282304
PipelineJob resource projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-jobstate-component-20250213223849 deleted.


In [92]:
try:
    jobs_list[0].state
except:
    print('Job not found - it was deleted!')

Job not found - it was deleted!


### Clone A Job To Run It Again

In [101]:
pipeline_job2.name

'mlops-pipeline-jobstate-component-20250213214746'

In [93]:
pipeline_job2.state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [97]:
clone_job2 = pipeline_job2.clone()

In [98]:
type(clone_job2)

google.cloud.aiplatform.pipeline_jobs.PipelineJob

In [99]:
clone_job2.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/cloned-mlops-pipeline-jobstate-component-20250213224824
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/cloned-mlops-pipeline-jobstate-component-20250213224824')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/cloned-mlops-pipeline-jobstate-component-20250213224824?project=1026793852137


In [100]:
clone_job2.name

'cloned-mlops-pipeline-jobstate-component-20250213224824'

In [115]:
clone_job2.wait()

PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/cloned-mlops-pipeline-jobstate-component-20250213224824


In [116]:
clone_job2.done()

True